In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
np.set_printoptions(threshold=np.inf, suppress=True)
import matplotlib.pyplot as plt

Loading data

Dataset obsahuje 60 continues features a jeden predicate (R/M), ktorý vraví o tom, či sa jedná o kameň alebo mínu. 

In [ ]:
sonar_data = pd.read_csv('dataset/sonar.all-data', header=None)
print(sonar_data.head())
print(sonar_data.shape)
print(sonar_data[60].value_counts())

In [ ]:
sonar_data.describe()

Vykonanie encodingu, kde mínu zakódujeme ako 1 a kameň ako 0

In [ ]:
# replacnutie R za 0 a M za 1
# R - Rock M - Mina
sonar_data[60] = sonar_data[60].replace(['R', 'M'], [0, 1])
sonar_data.head()

Data normalization

Data, ktoré používame normalizujeme pomocou standard scaleru

In [ ]:
normalized_dfMinMax = sonar_data.copy()
for x in range(60):
    normalized_dfMinMax[x] = MinMaxScaler().fit_transform(np.array(normalized_dfMinMax[x]).reshape(-1,1))

normalized_dfMinMax

normalized_df = sonar_data.copy()
for x in range(60):
    normalized_df[x] = StandardScaler().fit_transform(np.array(normalized_df[x]).reshape(-1,1))

normalized_df


In [ ]:
normalized_df.describe()

Train test val split 80/10/10

Dáta delíme takto kvôli menšiemu počtu dostupných vzoriek

In [ ]:
X = normalized_df.drop(columns=60, axis=1)
y = normalized_df[60]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

In [ ]:
print(X_train.shape)
print(y_train.value_counts())
print('******')
print(X_test.shape)
print(y_test.value_counts())
print('******')
print(X_val.shape)
print(y_val.value_counts())

Tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
import wandb
from wandb.keras import WandbCallback

Vytvorenie modelu

V rámci tensorflow časti vytvárame 5 rôznych modelov

Všetky majú rovnakú architektúru, len sa líšia v metódach použitých pri riešení problému overfittingu. Máme vstupnú vrstvu so 16 neuronmi, dve skryté vrstvy s 16 neurónmi a nakoniec výstupnú vrstvu s jedným neurónom.

Pri všetkých vrstvách používame aktivačnú funkciu ReLu až na poslednú, kde používame sigmoid.

Model 1 nepoužíva žiadnu metódu prevencie overfittingu

Model 2 používa metódu dropout, kde sme ako šancu vypadnuta neurónu dali 50%, dropout sa vykonáva pri prvej a druhej skrytej vrstve.

Model 3 používa early stopping, riadi sa podľa val_loss a parameter patience sme nastavili na 10 epoch.

Model 4 používa regularizáciu, ako hodnotu parametrov l1 a l2 sme zvolili hodnotu 0,005. Skúsili sme použiť aj vyššie hodnoty ale to viedlo k oveľa horším výsledkom, kde napríklad acc padlo približne na 55%.

Model 5 sme skúsili otestovať ako sa bude NN správať ak použijeme naraz dropout a early stopping, parametre sme použili rovnaké ako pri predchádzajúcich modeloch.





In [ ]:
#nothing
tf_model1 = Sequential()
tf_model1.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model1.add(Dense(16, activation=tf.keras.activations.relu))
tf_model1.add(Dense(16, activation=tf.keras.activations.relu))
tf_model1.add(Dense(1, activation=tf.keras.activations.sigmoid))
#dropout
tf_model2 = Sequential()
tf_model2.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model2.add(Dropout(0.5))
tf_model2.add(Dense(16, activation=tf.keras.activations.relu))
tf_model2.add(Dropout(0.5))
tf_model2.add(Dense(16, activation=tf.keras.activations.relu))
tf_model2.add(Dense(1, activation=tf.keras.activations.sigmoid))
#early stopping
tf_model3 = Sequential()
tf_model3.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model3.add(Dense(16, activation=tf.keras.activations.relu))
tf_model3.add(Dense(16, activation=tf.keras.activations.relu))
tf_model3.add(Dense(1, activation=tf.keras.activations.sigmoid))
earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
#regularization
tf_model4 = Sequential()
tf_model4.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns), kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.005)))
tf_model4.add(Dense(16, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.005)))
tf_model4.add(Dense(16, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.005)))
tf_model4.add(Dense(1, activation=tf.keras.activations.sigmoid))
#dropout + earlystop
tf_model5 = Sequential()
tf_model5.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model5.add(Dropout(0.5))
tf_model5.add(Dense(16, activation=tf.keras.activations.relu))
tf_model5.add(Dropout(0.5))
tf_model5.add(Dense(16, activation=tf.keras.activations.relu))
tf_model5.add(Dense(1, activation=tf.keras.activations.sigmoid))



Ako loss funkciu sme zvolili binary crossentropy a ako optimizer Adam.

Na sledovanie NN sme vybrali 4 metriky Acc, MSE, precision a recall

In [ ]:
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
tf_model1.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model2.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model3.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model4.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model5.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])

Batch size sme zvolili na 16.

Počet epoch trénovania sme zvolili 100, pri tomto zadaní by mal postačovať aj menší počet epoch ale kvôli krátkemu času behu sme nechali 100 epoch.

In [69]:
def fitModel(model, name, callbackArg):
    
    model.fit(
    X_train,
    y_train,
    batch_size=16,
    epochs=100,
    validation_data=(X_val,y_val),
    callbacks=callbackArg
    )

    model.save('./'+ name)
    

In [70]:

w = wandb.init(project='zadanie1',reinit=True)
w.config.epochs = 100
w.config.batch_size = 16



fitModel(tf_model1, 'model1', [WandbCallback()])

accuracy,▃▅▇███████▁▃▃▄▄▅▅▆▆▆▇▆▇▆▇▇▇▇█▇██▇█▇▇████
epoch,▁▂▃▃▄▅▆▇▇█▂▂▃▄▅▅▆▇█▁▂▁▂▂▃▄▅▆▆▇█▂▂▃▄▅▅▆▇█
loss,▇▅▃▂▂▁▁▁▁▁█▇▇▆▆▅▅▄▄▆▃▄▃▄▃▃▂▂▂▃▂▂▂▁▂▂▂▂▂▁
mse,▇▅▃▂▁▁▁▁▁▁█▇▇▆▅▅▅▄▄▅▂▃▃▄▃▃▂▂▁▂▂▂▂▁▂▂▁▂▁▁
precision_2,▂▅████████▁▃▃▄▄▅▅▆▆▆▇▆▇▆▇▇▇▇▇▇▇█▇█▇▇█▇██
recall_2,▇▆▆███████▁▅▄▅▅▅▅▆▆▇▇▇▆▇▇▇███▇█████▇████
val_accuracy,▁█▆███████▁▁▃▁▃▃▆▆▃▁█▆▆▆▆▃▆▆▆▆▆▆▆▆▆▆▆▆▆▆
val_loss,▂▁▁▁▂▂▂▃▃▃▂▂▂▁▁▁▂▂▂▂▁▂▃▃▄▄▄▅▅▅▅▆▆▆▇█▇█▇█
val_mse,█▂▁▃▃▄▄▅▅▅▇▆▄▃▄▃▃▃▃▇▂▄▄▄▅▆▅▆▇▆▇██▇▇▇▇█▇█
val_precision_2,▁▆▅▅▅▅▅▅▅▅▂▂▃▂▄▄▇▇▄▂█▇▇▅▅▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅
val_recall_2,█▆▃███████▁▃▃▁▁▁▁▁▁▃▃▁▁▃▃▁▃▃▃▃▃▃▃▃▃▃▃▃▃▃


Epoch 1/100
 1/11 [=>............................] - ETA: 0s - loss: 0.0012 - accuracy: 1.0000 - mse: 7.0446e-06 - precision_2: 1.0000 - recall_2: 1.0000WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0016s vs `on_train_batch_end` time: 0.0018s). Check your callbacks.


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_022825-h7pawtdz\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_022825-h7pawtdz\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_022825-h7pawtdz\files\model-best)... Done. 0.0s


11/11 [==============================] - 1s 108ms/step - loss: 0.0018 - accuracy: 1.0000 - mse: 2.3352e-05 - precision_2: 1.0000 - recall_2: 1.0000 - val_loss: 0.8640 - val_accuracy: 0.8095 - val_mse: 0.1847 - val_precision_2: 0.6923 - val_recall_2: 1.0000
Epoch 2/100
 1/11 [=>............................] - ETA: 0s - loss: 7.9037e-04 - accuracy: 1.0000 - mse: 2.1706e-06 - precision_2: 1.0000 - recall_2: 1.0000

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_022825-h7pawtdz\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_022825-h7pawtdz\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_022825-h7pawtdz\files\model-best)... Done. 0.0s


11/11 [==============================] - 1s 110ms/step - loss: 0.0018 - accuracy: 1.0000 - mse: 2.2044e-05 - precision_2: 1.0000 - recall_2: 1.0000 - val_loss: 0.8632 - val_accuracy: 0.8095 - val_mse: 0.1846 - val_precision_2: 0.6923 - val_recall_2: 1.0000
Epoch 3/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0017 - accuracy: 1.0000 - mse: 2.0471e-05 - precision_2: 1.0000 - recall_2: 1.0000 - val_loss: 0.8639 - val_accuracy: 0.8095 - val_mse: 0.1844 - val_precision_2: 0.6923 - val_recall_2: 1.0000
Epoch 4/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0016 - accuracy: 1.0000 - mse: 1.8760e-05 - precision_2: 1.0000 - recall_2: 1.0000 - val_loss: 0.8678 - val_accuracy: 0.8095 - val_mse: 0.1846 - val_precision_2: 0.6923 - val_recall_2: 1.0000
Epoch 5/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0016 - accuracy: 1.0000 - mse: 1.7051e-05 - precision_2: 1.0000 - recall_2: 1.0000 - val_loss: 0.8685 - val_accuracy: 0.8095 - val_mse

INFO:tensorflow:Assets written to: ./model1\assets


INFO:tensorflow:Assets written to: ./model1\assets


In [71]:
fitModel(tf_model2, 'model2', [WandbCallback()])

Epoch 1/100
11/11 [==============================] - 0s 10ms/step - loss: 0.0263 - accuracy: 0.9940 - mse: 0.0064 - precision_2: 1.0000 - recall_2: 0.9890 - val_loss: 2.1222 - val_accuracy: 0.7619 - val_mse: 0.2226 - val_precision_2: 0.7000 - val_recall_2: 0.7778
Epoch 2/100
11/11 [==============================] - 0s 5ms/step - loss: 0.0623 - accuracy: 0.9819 - mse: 0.0185 - precision_2: 0.9681 - recall_2: 1.0000 - val_loss: 2.1139 - val_accuracy: 0.7619 - val_mse: 0.2229 - val_precision_2: 0.7000 - val_recall_2: 0.7778
Epoch 3/100
11/11 [==============================] - 0s 5ms/step - loss: 0.0556 - accuracy: 0.9819 - mse: 0.0160 - precision_2: 0.9681 - recall_2: 1.0000 - val_loss: 2.0593 - val_accuracy: 0.7619 - val_mse: 0.2223 - val_precision_2: 0.7000 - val_recall_2: 0.7778
Epoch 4/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0974 - accuracy: 0.9639 - mse: 0.0303 - precision_2: 0.9474 - recall_2: 0.9890 - val_loss: 1.9347 - val_accuracy: 0.7619 - val_mse: 0.2

INFO:tensorflow:Assets written to: ./model2\assets


INFO:tensorflow:Assets written to: ./model2\assets


In [72]:
w.finish()
w = wandb.init(project='zadanie1',reinit=True)

accuracy,████████████████████▄▇▁▂█▂▅▂▇▅▄█▅▇▅▄▇▅▅▄
epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇█▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇██
loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▄▇▆▃█▄▆▄▄▅▃▄▃▄▅▃▃▄▅
mse,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▄█▇▂█▄▇▃▄▅▂▄▃▄▅▃▃▄▅
precision_2,████████████████████▃▆▁▁█▁▄▁▆▄▃█▆▆▄▃▆▄▄▆
recall_2,██████████████████████▅█████████▅██████▁
val_accuracy,████████████████████▄▄▄▄▄▄▁▁▁▁▁▁▁▁▁▁▁▄▄▄
val_loss,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▆▅▆▇▇▇▇▇▇▇▇▇███▇▇▇▇▇
val_mse,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▃▄▄▄▅▆▆███████▅▅▄▂▄
val_precision_2,▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▁▁▁▁▁▁▁▁▁▁▁█▄▄
val_recall_2,████████████████████▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▃▃


In [73]:
fitModel(tf_model3, 'model3', [earlyStop,WandbCallback()])

Epoch 1/100
 1/11 [=>............................] - ETA: 0s - loss: 0.0375 - accuracy: 1.0000 - mse: 0.0031 - precision_2: 1.0000 - recall_2: 1.0000

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_022953-pk2upxbl\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_022953-pk2upxbl\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_022953-pk2upxbl\files\model-best)... Done. 0.0s


11/11 [==============================] - 1s 106ms/step - loss: 0.0480 - accuracy: 0.9940 - mse: 0.0087 - precision_2: 1.0000 - recall_2: 0.9890 - val_loss: 0.5512 - val_accuracy: 0.7619 - val_mse: 0.1638 - val_precision_2: 0.7000 - val_recall_2: 0.7778
Epoch 2/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0434 - accuracy: 0.9940 - mse: 0.0076 - precision_2: 1.0000 - recall_2: 0.9890 - val_loss: 0.5684 - val_accuracy: 0.7619 - val_mse: 0.1662 - val_precision_2: 0.7000 - val_recall_2: 0.7778
Epoch 3/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0391 - accuracy: 0.9940 - mse: 0.0065 - precision_2: 1.0000 - recall_2: 0.9890 - val_loss: 0.5888 - val_accuracy: 0.7619 - val_mse: 0.1669 - val_precision_2: 0.7000 - val_recall_2: 0.7778
Epoch 4/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0349 - accuracy: 0.9940 - mse: 0.0055 - precision_2: 1.0000 - recall_2: 0.9890 - val_loss: 0.6041 - val_accuracy: 0.7619 - val_mse: 0.1679 - val_p

INFO:tensorflow:Assets written to: ./model3\assets


INFO:tensorflow:Assets written to: ./model3\assets


In [74]:
w.finish()
w = wandb.init(project='zadanie1',reinit=True)

accuracy,▁▁▁▁▁▁█████
epoch,▁▂▂▃▄▅▅▆▇▇█
loss,█▇▆▅▄▃▃▂▂▁▁
mse,█▇▆▅▄▃▂▂▂▁▁
precision_2,▁▁▁▁▁▁▁▁▁▁▁
recall_2,▁▁▁▁▁▁█████
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▂▃▄▄▅▆▇▇▇█
val_mse,▁▃▄▅▅▄▅▅█▆▅
val_precision_2,▁▁▁▁▁▁▁▁▁▁▁
val_recall_2,▁▁▁▁▁▁▁▁▁▁▁


In [75]:
fitModel(tf_model4, 'model4', [WandbCallback()])

Epoch 1/100
 1/11 [=>............................] - ETA: 10s - loss: 2.2854 - accuracy: 0.3125 - mse: 0.2808 - precision_2: 0.6923 - recall_2: 0.4286WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0013s vs `on_train_batch_end` time: 0.0017s). Check your callbacks.


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023429-v31cbnnt\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023429-v31cbnnt\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023429-v31cbnnt\files\model-best)... Done. 0.0s


11/11 [==============================] - 3s 185ms/step - loss: 2.1851 - accuracy: 0.5904 - mse: 0.2418 - precision_2: 0.6627 - recall_2: 0.5500 - val_loss: 2.1319 - val_accuracy: 0.5238 - val_mse: 0.2305 - val_precision_2: 0.4545 - val_recall_2: 0.5556
Epoch 2/100
 1/11 [=>............................] - ETA: 0s - loss: 2.1815 - accuracy: 0.6250 - mse: 0.2524 - precision_2: 1.0000 - recall_2: 0.5000

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023429-v31cbnnt\files\model-best\assets


...

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023429-v31cbnnt\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023429-v31cbnnt\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023429-v31cbnnt\files\model-best)... Done. 0.0s


11/11 [==============================] - 1s 146ms/step - loss: 0.2430 - accuracy: 1.0000 - mse: 0.0023 - precision_2: 1.0000 - recall_2: 1.0000 - val_loss: 0.7851 - val_accuracy: 0.8095 - val_mse: 0.1593 - val_precision_2: 0.7273 - val_recall_2: 0.8889


INFO:tensorflow:Assets written to: ./model4\assets


INFO:tensorflow:Assets written to: ./model4\assets


In [77]:
w.finish()
w = wandb.init(project='zadanie1',reinit=True)

In [ ]:
fitModel(tf_model5, 'model5', [earlyStop,WandbCallback()])

Epoch 1/100
 1/11 [=>............................] - ETA: 9s - loss: 0.6934 - accuracy: 0.5625 - mse: 0.2523 - precision_2: 0.6522 - recall_2: 0.8333WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0018s vs `on_train_batch_end` time: 0.0024s). Check your callbacks.


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023819-four8km7\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023819-four8km7\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023819-four8km7\files\model-best)... Done. 0.0s


11/11 [==============================] - 2s 141ms/step - loss: 0.7413 - accuracy: 0.5301 - mse: 0.2632 - precision_2: 0.5789 - recall_2: 0.6600 - val_loss: 0.6921 - val_accuracy: 0.5238 - val_mse: 0.2490 - val_precision_2: 0.4737 - val_recall_2: 1.0000
Epoch 2/100
 1/11 [=>............................] - ETA: 0s - loss: 0.5040 - accuracy: 0.6875 - mse: 0.1652 - precision_2: 0.5556 - recall_2: 0.8333

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023819-four8km7\files\model-best\assets


...

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023819-four8km7\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023819-four8km7\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023819-four8km7\files\model-best)... Done. 0.0s


11/11 [==============================] - 1s 123ms/step - loss: 0.6216 - accuracy: 0.6084 - mse: 0.2180 - precision_2: 0.6121 - recall_2: 0.7802 - val_loss: 0.6130 - val_accuracy: 0.5714 - val_mse: 0.2138 - val_precision_2: 0.5000 - val_recall_2: 0.8889
Epoch 19/100
 1/11 [=>............................] - ETA: 0s - loss: 0.5871 - accuracy: 0.7500 - mse: 0.1973 - precision_2: 0.7778 - recall_2: 0.7778

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023819-four8km7\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023819-four8km7\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_023819-four8km7\files\model-best)... 

In [ ]:
w.finish()
w = wandb.init(project='zadanie1',reinit=True)

In [ ]:
#pomocna funkcia
def predictEval(tf_model, XX, yy):
    # vykonanie predikcie
    y_pred = tf_model.predict(XX)
    # uprava outputu na boolean
    y_pred_bool = np.copy(y_pred)
    for x in y_pred_bool:
        x[0] = round(x[0])
    y_pred_bool

    #vratenie y a accuaracy
    return [y_pred, y_pred_bool, accuracy_score(y_pred_bool, yy), mean_squared_error(y_pred_bool, yy), precision_score(y_pred_bool, yy), recall_score(y_pred_bool, yy)]

Eval

In [ ]:
def predictEvalWrap(model, name):
    train = predictEval(model, X_train, y_train)
    val = predictEval(model, X_val, y_val)
    test = predictEval(model, X_test, y_test)

    print(name)
    print('Accuracy score')
    print(f'Train: {train[2]*100:.2f}%')
    print(f'Val: {val[2]*100:.2f}%')
    print(f'Test: {test[2]*100:.2f}%')
    print('Mean squared error')
    print(f'Train: {train[3]*100:.2f}%')
    print(f'Val: {val[3]*100:.2f}%')
    print(f'Test: {test[3]*100:.2f}%')
    print('Precision')
    print(f'Train: {train[4]*100:.2f}%')
    print(f'Val: {val[4]*100:.2f}%')
    print(f'Test: {test[4]*100:.2f}%')
    print('Recall')
    print(f'Train: {train[5]*100:.2f}%')
    print(f'Val: {val[5]*100:.2f}%')
    print(f'Test: {test[5]*100:.2f}%')
    print('------------------------------------------')

In [ ]:
predictEvalWrap(tf_model1, 'No overfit prevention')
predictEvalWrap(tf_model2, 'Dropout')
predictEvalWrap(tf_model3, 'Early stopping')
predictEvalWrap(tf_model4, 'Regularization')
predictEvalWrap(tf_model5, 'Dropout + Early stopping')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

In [ ]:
# Define the PyTorch model
class PyTorchModel(nn.Module):
    def __init__(self, input_dim):
        super(PyTorchModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

In [ ]:
class PyTorchModelDropout(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.5):
        super(PyTorchModelDropout, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(16, 16)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(16, 16)
        self.dropout3 = nn.Dropout(dropout_rate)
        self.fc4 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout3(x)
        x = torch.sigmoid(self.fc4(x))
        return x

In [ ]:
class SonarDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_dataset = SonarDataset(X_train, y_train)
val_dataset = SonarDataset(X_val, y_val)
test_dataset = SonarDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [ ]:
# Instantiate the model
input_dim = len(X_train.columns)
model = PyTorchModel(input_dim)
model_dropout = PyTorchModelDropout(input_dim, dropout_rate=0.5)
model_early_stopping = PyTorchModel(input_dim)

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
optimizer_dropout = optim.Adam(model_dropout.parameters())
optimizer_early_stopping = optim.Adam(model_early_stopping.parameters())

In [ ]:
num_epochs = 100

In [ ]:

for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        wandb.log({"loss": loss})
    with torch.no_grad():
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            y_val_pred = model(X_val_batch)
            val_loss = criterion(y_val_pred, y_val_batch)

    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

In [ ]:
for epoch in range(num_epochs):
    model_dropout.train()
    for X_batch, y_batch in train_loader:
        optimizer_dropout.zero_grad()
        y_pred = model_dropout(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        model_dropout.eval()
        val_loss_sum = 0
        val_batch_count = 0
        for X_val_batch, y_val_batch in val_loader:
            y_val_pred = model_dropout(X_val_batch)
            val_loss = criterion(y_val_pred, y_val_batch)
            val_loss_sum += val_loss.item()
            val_batch_count += 1
        val_loss_avg = val_loss_sum / val_batch_count
    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss_avg:.4f}")

In [ ]:
patience = 10

best_val_loss = float("inf")
counter = 0

for epoch in range(num_epochs):
    model_early_stopping.train()
    for X_batch, y_batch in train_loader:
        optimizer_early_stopping.zero_grad()
        y_pred = model_early_stopping(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        model_early_stopping.eval()
        val_loss_sum = 0
        val_batch_count = 0
        for X_val_batch, y_val_batch in val_loader:
            y_val_pred = model_early_stopping(X_val_batch)
            val_loss = criterion(y_val_pred, y_val_batch)
            val_loss_sum += val_loss.item()
            val_batch_count += 1
        val_loss_avg = val_loss_sum / val_batch_count

    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss_avg:.4f}")

    # Early stopping logic
    if val_loss_avg < best_val_loss:
        best_val_loss = val_loss_avg
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping!")
            break

In [ ]:
def predict_eval(model, loader):
    with torch.no_grad():
        model.eval()
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            y_pred_bool = torch.round(y_pred)
            accuracy = accuracy_score(y_pred_bool, y_batch)
    return accuracy

In [ ]:
train_accuracy = predict_eval(model, train_loader)
val_accuracy = predict_eval(model, val_loader)
test_accuracy = predict_eval(model, test_loader)

In [ ]:
train_accuracy_dropout = predict_eval(model_dropout, train_loader)
val_accuracy_dropout = predict_eval(model_dropout, val_loader)
test_accuracy_dropout = predict_eval(model_dropout, test_loader)

In [ ]:
train_accuracy_early_stopping = predict_eval(model_early_stopping, train_loader)
val_accuracy_early_stopping = predict_eval(model_early_stopping, val_loader)
test_accuracy_early_stopping = predict_eval(model_early_stopping, test_loader)

In [ ]:
print('No overfit prevention')
print(f'Train: {train_accuracy * 100:.2f}%')
print(f'Val: {val_accuracy * 100:.2f}%')
print(f'Test: {test_accuracy * 100:.2f}%')

In [ ]:
print('Droupout')
print(f'Train: {train_accuracy_dropout * 100:.2f}%')
print(f'Val: {val_accuracy_dropout * 100:.2f}%')
print(f'Test: {test_accuracy_dropout * 100:.2f}%')

In [ ]:
print('Early stopping')
print(f'Train: {train_accuracy_early_stopping * 100:.2f}%')
print(f'Val: {val_accuracy_early_stopping * 100:.2f}%')
print(f'Test: {test_accuracy_early_stopping * 100:.2f}%')